In [15]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

In [ ]:
# Paths
train_dir = "../../data4/train"
validation_dir = "../../data4/validation"
test_dir = "../../data4/test"
model_path = "../../models/improved_hybrid_model.keras"

In [ ]:
# Load your trained CNN model
cnn_model = load_model(model_path)


In [18]:
# Extract features from the second-last layer
feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)


In [ ]:

# Data Generator
datagen = ImageDataGenerator(rescale=1.0 / 255.0)  # Normalize pixel values

# Data Generators
train_generator = datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode="categorical", shuffle=True
)
validation_generator = datagen.flow_from_directory(
    validation_dir, target_size=(224, 224), batch_size=32, class_mode="categorical", shuffle=False
)
test_generator = datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=32, class_mode="categorical", shuffle=False
)


Found 4000 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [20]:
# Function to Extract Features
def extract_features(generator, feature_extractor):
    features = []
    labels = []

    for batch_images, batch_labels in generator:
        batch_features = feature_extractor.predict(batch_images)
        features.append(batch_features)
        labels.append(batch_labels)

        # Stop when all data is processed
        if generator.batch_index == 0:
            break

    return np.vstack(features), np.vstack(labels)


# Extract Features for Train, Validation, and Test Data
train_features, train_labels = extract_features(train_generator, feature_extractor)
validation_features, validation_labels = extract_features(validation_generator, feature_extractor)
test_features, test_labels = extract_features(test_generator, feature_extractor)

# Save features for reuse
np.save("train_features.npy", train_features)
np.save("train_labels.npy", train_labels)
np.save("test_features.npy", test_features)
np.save("test_labels.npy", test_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 

In [21]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)


# Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predict and Evaluate Random Forest
y_pred_rf = rf_model.predict(X_test)
print("\nRandom Forest Classifier Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.2f}")
print(classification_report(y_test, y_pred_rf))


# Feature Scaling for KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train KNN Classifier
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Predict and Evaluate KNN
y_pred_knn = knn_model.predict(X_test_scaled)
print("\nKNN Classifier Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")
print(classification_report(y_test, y_pred_knn))


Random Forest Classifier Performance:
Accuracy: 0.73
              precision    recall  f1-score   support

           0       0.81      0.57      0.67       209
           1       0.79      0.71      0.75       193
           2       0.91      0.83      0.87       191
           3       0.98      0.82      0.89       201

   micro avg       0.87      0.73      0.80       794
   macro avg       0.87      0.73      0.79       794
weighted avg       0.87      0.73      0.79       794
 samples avg       0.73      0.73      0.73       794



/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



KNN Classifier Performance:
Accuracy: 0.74


/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.70      0.57      0.63       209
           1       0.69      0.68      0.69       193
           2       0.85      0.85      0.85       191
           3       0.96      0.85      0.90       201

   micro avg       0.80      0.74      0.77       794
   macro avg       0.80      0.74      0.77       794
weighted avg       0.80      0.74      0.77       794
 samples avg       0.74      0.74      0.74       794



In [1]:
import tensorflow as tf
print(tf.__version__)


2.15.0


In [2]:
from tensorflow.keras.models import load_model

# Path to the saved model
model_path = "../../colabModels/hybrid_model_vgg16_tpu.keras"

# Load the model
model = load_model(model_path)
print("Model loaded successfully.")

Model loaded successfully.


In [3]:
model.save("../../models/tpu_vgg16.keras")

In [1]:
import os
import numpy as np
from flask import Flask, request, render_template, url_for
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from werkzeug.utils import secure_filename
import joblib
import re
from tensorflow.keras.applications import ResNet50

# Flask application setup
app = Flask(__name__, template_folder="templates", static_folder="src/static")

# Model paths
MODEL_PATHS = {
    "vgg16": "../../models/tpu_vgg16.keras",
    "resnet50": "colabModels/hybrid_model_resnet50_fixed.keras",
}

# Load models and utilities
# cnn_model = load_model(MODEL_PATHS["cnn"])
vgg16_model = load_model(MODEL_PATHS["vgg16"])
resnet50_model = load_model(MODEL_PATHS["resnet50"])
# inceptionv3_model = load_model(MODEL_PATHS["inceptionv3"])
# rf_model = joblib.load(MODEL_PATHS["rf"])
# res_rf_model = joblib.load(MODEL_PATHS["res_rf"])
# pca = joblib.load(MODEL_PATHS["pca"])
# scaler = joblib.load(MODEL_PATHS["scaler"])
# resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Labels and medication suggestions
LABELS = ["Acne", "Eczema", "Fungal Infection", "Malignant"]
MEDICATIONS = {
    0: "Consult a dermatologist. Oral antibiotics might be required.",
    1: "Use hydrocortisone cream to reduce inflammation. Avoid allergens.",
    2: "Apply topical antifungal cream. Keep the area clean and dry.",
    3: "Moisturize regularly. Use over-the-counter emollients."
}

# Utility functions
def preprocess_image(image_path, target_size):
    """Preprocess the input image to match the model's required input size."""
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

def predict_with_cnn(model, image_path, target_size):
    """Predict using a CNN model after resizing the image to the required size."""
    img_array = preprocess_image(image_path, target_size)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    confidence = float(np.max(predictions)) * 100
    return predicted_class, confidence

def extract_features_with_cnn(cnn_model, image_path, target_size):
    """Extract features from the CNN's intermediate layer."""
    img_array = preprocess_image(image_path, target_size)
    feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
    features = feature_extractor.predict(img_array)
    features_flat = features.reshape(1, -1)
    return features_flat.flatten()

def adjust_feature_dimensions(features, expected_dim):
    """Adjust feature dimensions to match the expected size for PCA."""
    current_dim = features.shape[0]
    if current_dim < expected_dim:
        padding = np.zeros(expected_dim - current_dim)
        return np.hstack((features, padding))
    return features[:expected_dim]

def apply_pca(features, pca_model, expected_dim=41472):
    """Apply PCA transformation to reduce feature dimensions."""
    adjusted_features = adjust_feature_dimensions(features, expected_dim)
    return pca_model.transform(adjusted_features.reshape(1, -1))

def predict_with_rf(image_path, cnn_model, rf_model, pca_model, target_size, expected_dim=41472):
    """Predict using RandomForest with features extracted from a CNN model."""
    features = extract_features_with_cnn(cnn_model, image_path, target_size)
    features = apply_pca(features, pca_model, expected_dim)
    return rf_model.predict(features)[0]

resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
def predict_disease_by_tdm(image_path, model, scaler_path=None, class_labels=None):
    """
    Predict the disease from an image using a trained model (KNN or Random Forest).
    
    Args:
        image_path (str): Path to the image.
        model: Trained classification model (KNN or Random Forest).
        scaler: Scaler used to normalize features (optional, for KNN).
        class_labels (dict): Mapping of class indices to class names.
        
    Returns:
        prediction (str): Predicted class label (name).
    """
    # Load and preprocess the image
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # Extract features using ResNet50
    resnet_features = resnet.predict(img_array)
    resnet_features_flat = resnet_features.reshape(1, -1)
    
    # # Load scaler if path is provided and scale features
    # if scaler_path:
    #     scaler = load(scaler_path)
    #     resnet_features_flat = scaler.transform(resnet_features_flat)
    
    # Make a prediction
    prediction = model.predict(resnet_features_flat)
    predicted_class_index = prediction[0]  # Extract the predicted class index
    
    # Map class index to class name
    if class_labels:
        predicted_class = class_labels.get(predicted_class_index, "Unknown Class")
    else:
        predicted_class = str(predicted_class_index)  # Fallback to index if no mapping provided
    
    return predicted_class

# Flask routes
@app.route('/', methods=['GET', 'POST'])
def upload_predict():
    if request.method == 'POST':
        image_file = request.files['file']
        if image_file:
            upload_dir = "src/static/uploads"
            os.makedirs(upload_dir, exist_ok=True)
            filename = secure_filename(image_file.filename)
            filename = re.sub(r'[^a-zA-Z0-9_.-]', '_', filename)
            image_location = os.path.join(upload_dir, filename)
            image_file.save(image_location)

            # Predictions
            vgg_pred, vgg_conf = predict_with_cnn(vgg16_model, image_location, target_size=(224, 224))
            res_pred, res_conf = predict_with_cnn(resnet50_model, image_location, target_size=(224, 224))
            # rf_pred = predict_with_rf(image_location, vgg16_model, rf_model, pca, target_size=(299, 299), expected_dim=41472)
            # res_rf_pred = predict_with_rf(image_location, resnet50_model, res_rf_model, pca, target_size=(224, 224), expected_dim=41472)

            # rf_trandition =  predict_disease_by_tdm(image_location,res_rf_model)

            # Find the Best Model
            model_confidences = {
                # "Hybrid Model": hybrid_confidence,
                "VGG16": vgg_conf,
                "ResNet50": res_conf,
                # "InceptionV3": inceptionv3_confidence
            }
            best_model = max(model_confidences, key=model_confidences.get)

            # Render result
            return render_template(
                'result.html',
                vgg16_label=LABELS[vgg_pred],
                resnet50_label=LABELS[res_pred],
                # rf_label=LABELS[rf_trandition],
                # rf_pred=res_rf_model,
                model_confidences=model_confidences,
                medication=MEDICATIONS.get(vgg_pred, "Consult a dermatologist."),
                image_url=url_for('static', filename=f'uploads/{filename}')
            )
    return render_template('index.html')

if __name__ == '__main__':
    os.makedirs("src/static/uploads", exist_ok=True)
    os.makedirs("templates", exist_ok=True)
    app.run(debug=True, host='0.0.0.0', port=5030)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5030
 * Running on http://192.168.1.64:5030
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
Traceback (most recent call last):
  File "/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/Users/liraj/.pyenv/vers

SystemExit: 1

/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [6]:
exit

: 